In [ ]:
%pip install speechrecognition pyaudio
%pip install pyttsx3

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, TimeDistributed
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import speech_recognition as sr
import ipywidgets as widgets
from IPython.display import display, clear_output
import pyttsx3

print("TensorFlow Version:", tf.__version__)

# Initial dataset
data = {
    "questions": [        
        "What are your operating hours?",
        "Do you offer vegan options?",
        "Is there outdoor seating?",
        "Can I make a reservation?",
        "What is the Wi-Fi password?",
        "Do you have parking available?",
        "Can I place an order for pickup?",
        "Do you offer delivery services?",
        "What is your refund policy?",
        "Are pets allowed?",
    ],
    "responses": [
        "We are open from 8 AM to 10 PM.",
        "Yes, we offer a variety of vegan dishes.",
        "Yes, we have a beautiful outdoor seating area.",
        "Yes, you can make a reservation through our website or by calling us.",
        "The Wi-Fi password is 'guest123'.",
        "Yes, we have parking available for our customers.",
        "Yes, you can place an order for pickup through our website.",
        "Yes, we offer delivery through our app.",
        "You can request a refund within 24 hours of your purchase.",
        "Yes, pets are allowed in the outdoor seating area.",
    ]
}

# Add additional questions and responses
additional_data = {
    "questions": [
        "Do you offer gluten-free options?",
        "Can I customize my order?",
        "What is your most popular dish?",
        "Are there any new items on the menu?",
        "Can I schedule a delivery for later?",
        "Do you have any combo offers?",
        "What are your payment options?",
        "Can I cancel my order?",
        "Is there a minimum order amount?",
        "How can I give feedback on my order?"
    ],
    "responses": [
        "Yes, we offer gluten-free pasta and pizza.",
        "Yes, you can customize your order through our app or website.",
        "Our most popular dish is the margherita pizza.",
        "Yes, we have recently added sushi to our menu.",
        "Yes, you can schedule a delivery time while placing your order.",
        "Yes, we have combo offers available in the app.",
        "You can pay via credit card, debit card, or mobile payments.",
        "Yes, you can cancel your order within 10 minutes of placing it.",
        "There is no minimum order amount.",
        "You can give feedback through our app or website.",
    ]
}

# Combine the new data with the original dataset
data['questions'].extend(additional_data['questions'])
data['responses'].extend(additional_data['responses'])

# Update the dataframe
df = pd.DataFrame(data)

# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['questions'] + df['responses'])
vocab_size = len(tokenizer.word_index) + 1
max_len = max([len(seq) for seq in tokenizer.texts_to_sequences(df['questions'] + df['responses'])])

X = tokenizer.texts_to_sequences(df['questions'])
X = pad_sequences(X, maxlen=max_len, padding='post')

y = tokenizer.texts_to_sequences(df['responses'])
y = pad_sequences(y, maxlen=max_len, padding='post')

# Prepare Decoder Input Data
decoder_input_data = np.zeros((len(y), max_len, vocab_size))
for i, seq in enumerate(y):
    for t in range(len(seq) - 1):
        decoder_input_data[i, t, seq[t]] = 1

# Prepare Decoder Output Data
decoder_output_data = np.zeros((len(y), max_len, vocab_size))
for i, seq in enumerate(y):
    for t in range(len(seq)):
        decoder_output_data[i, t, seq[t]] = 1

# Build the Model
encoder_inputs = Input(shape=(max_len,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=128)(encoder_inputs)
encoder_lstm = LSTM(64, return_sequences=False)(encoder_embedding)
encoder_outputs = RepeatVector(max_len)(encoder_lstm)

decoder_inputs = Input(shape=(max_len, vocab_size))
decoder_lstm = LSTM(64, return_sequences=True)(decoder_inputs)
decoder_outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder_lstm)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the Model
history = model.fit([X, decoder_input_data], decoder_output_data, epochs=100, batch_size=8, validation_split=0.2)

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(df['questions'])
question_vectors = vectorizer.transform(df['questions'])

def get_closest_question(user_input):
    user_vector = vectorizer.transform([user_input])
    similarities = cosine_similarity(user_vector, question_vectors)
    closest_index = np.argmax(similarities)
    return df['responses'][closest_index]

# Text-to-Speech Engine
engine = pyttsx3.init()

def speak_text(text):
    engine.say(text)
    engine.runAndWait()

# Dropdown Menu for User Input
dropdown = widgets.Dropdown(
    options=df['questions'].tolist(),
    description='Select Question:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        response = get_closest_question(change['new'])
        print(f"Response: {response}")
        speak_text(response)
        display_restart_button()

dropdown.observe(on_change)

display(dropdown)

# Voice Recognition and Reply
def recognize_speech():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        print("Listening...")
        audio = recognizer.listen(source)

    try:
        user_input = recognizer.recognize_google(audio)
        print(f"User said: {user_input}")
        response = get_closest_question(user_input)
        print(f"Response: {response}")
        speak_text(response)
        display_restart_button()
    except sr.UnknownValueError:
        print("Sorry, I did not understand that.")
        speak_text("Sorry, I did not understand that.")
    except sr.RequestError:
        print("Sorry, there seems to be a request error.")
        speak_text("Sorry, there seems to be a request error.")

# Add a button for voice input
voice_button = widgets.Button(description="Voice Input")

def on_voice_button_click(b):
    recognize_speech()

voice_button.on_click(on_voice_button_click)
display(voice_button)

# Add a button to restart the chat
restart_button = widgets.Button(description="Restart Chat")

def on_restart_button_click(b):
    clear_output()
    display(dropdown)
    display(voice_button)

def display_restart_button():
    display(restart_button)

restart_button.on_click(on_restart_button_click)


Dropdown(description='Select Question:', index=12, options=('What are your operating hours?', 'Do you offer ve…

Button(description='Voice Input', style=ButtonStyle())

Response: Yes, you can schedule a delivery time while placing your order.


Button(description='Restart Chat', style=ButtonStyle())

Listening...
User said: Wi-Fi password
Response: The Wi-Fi password is 'guest123'.


Button(description='Restart Chat', style=ButtonStyle())

Listening...
User said: parking available
Response: Yes, we have parking available for our customers.


Button(description='Restart Chat', style=ButtonStyle())